In [ ]:
import pandas as pd
import mysql.connector
import numpy as np

In [ ]:
df_canada = pd.read_pickle('datos/df_canada.pkl')  ###abrimos los df con los datos limpios
df_argentina = pd.read_pickle('datos/df_argentina.pkl')
df_eeuu = pd.read_pickle('datos/df_eeuu.pkl')

In [ ]:
class Cargar:
    
    def __init__(self, nombre_bbdd, contraseña):
        # nuestra clase va a recibir dos parámetros, el nombre de la base de datos y la contraseña con el servidor. 
        self.nombre_bbdd = nombre_bbdd
        self.contraseña = contraseña
    
    def crear_bbdd(self):  ###funcion para crear la base de datos en caso que no la tengamos.

        mydb = mysql.connector.connect(host="localhost",
                                       user="root",
                                       password=f'{self.contraseña}') 
        mycursor = mydb.cursor()

        try:
            mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.nombre_bbdd};")
            
        except:
            print("La Base de Datos ya existe")
    
    def crear_insertar_tabla(self, query):
        
        mydb = mysql.connector.connect(host="localhost",
                                       user="root",
                                       password=f'{self.contraseña}', 
                                       database=f"{self.nombre_bbdd}") 
        mycursor = mydb.cursor()
        
        try:
            mycursor.execute(query)
            mydb.commit()
          
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)
    
    def check_provincia(self):
    
        mydb = mysql.connector.connect(user='root',
                                      password=f"{self.contraseña}",
                                      host='127.0.0.1',
                                      database=f"{self.nombre_bbdd}")
        mycursor = mydb.cursor()

        # query para extraer los valores únicos de provincias de la tabla paises
        query_existe_provincia = f"""
                SELECT DISTINCT nombre_provincia FROM paises
                """
        mycursor.execute(query_existe_provincia)
        provincia = mycursor.fetchall()
        return provincia

    def sacar_idestado(self, provincia):
        
        mydb = mysql.connector.connect(user='root',
                                       password= f'{self.contraseña}',
                                       host='127.0.0.1', 
                                       database=f"{self.nombre_bbdd}")
        mycursor = mydb.cursor()
        #### query para sacar los id de la tabla países, que los utilizaremos para crear la otra tabla
        try:
            query_sacar_id = f"SELECT idestado FROM paises WHERE nombre_provincia = '{provincia}'"
            mycursor.execute(query_sacar_id)
            id_ = mycursor.fetchall()[0][0]
            return id_
        
        except: 
            return "Sorry, no tenemos esa provincia en la BBDD y por lo tanto no te podemos dar su id. "
    

In [ ]:
query_tabla_paises = """
                    CREATE TABLE `universidades`.`paises` (
                        `idestado` INT NOT NULL AUTO_INCREMENT,
                        `nombre_pais` VARCHAR(100) NULL,
                        `nombre_provincia` VARCHAR(100) NULL,
                        `latitud` FLOAT NULL,
                        `longitud` FLOAT NULL,
                        PRIMARY KEY (`idestado`));
"""

In [ ]:
query_tabla_universidades = """
                            CREATE TABLE `universidades`.`universidades` (
                                `iduniversidades` INT NOT NULL AUTO_INCREMENT,
                                `nombre_universidad` VARCHAR(100) NULL,
                                `pagina_web` VARCHAR(100) NULL,
                                `paises_idestado` INT NULL,
                                PRIMARY KEY (`iduniversidades`),
                                CONSTRAINT `fk_universidades_paises` 
                                        FOREIGN KEY (`paises_idestado`) 
                                        REFERENCES `paises` (`idestado`) 
                                        ON DELETE CASCADE ON UPDATE CASCADE);
"""

In [ ]:
carga = Cargar('universidades', 'AlumnaAdalab')

In [ ]:
# tabla paises

def tabla_paises(df):
    for indice, fila in df.iterrows():
        
        # creamos la query que nos inserta datos en la tabla paises
        query_paises = f"""
                    INSERT INTO paises (nombre_pais, nombre_provincia, latitud, longitud) 
                    VALUES ( "{fila['country']}", "{fila['provincia']}", "{fila['latitud']}", "{fila['longitud']}");
                    """
        # llamamos a la funcion check_provincia para saber que provincias tenemos en la BBDD
        provincias = carga.check_provincia()
        
        if len(provincias) == 0 or fila["provincia"] not in provincias[0]: 

            carga.crear_insertar_tabla(query_paises)

        else:
            print(f"{fila['provincia']} ya esta en nuestra BBDD") 

In [ ]:
tabla_paises(df_argentina)

In [ ]:
tabla_paises(df_canada)

In [ ]:
tabla_paises(df_eeuu)

In [ ]:
# tabla universidades
def tabla_universidades(df): 

    for indice, fila in df.iterrows():

        id_pais = carga.sacar_idestado(fila['provincia'])  
        # creamos la query que nos inserta los datos en la tabla universidades
        query_universidades = f"""
                    INSERT INTO universidades (nombre_universidad, pagina_web, paises_idestado) 
                    VALUES ( "{fila['name']}", "{fila['web_pages']}", "{id_pais}");
                    """
        
        carga.crear_insertar_tabla(query_universidades)

In [ ]:
tabla_universidades(df_canada)

In [ ]:
tabla_universidades(df_argentina)

In [ ]:
tabla_universidades(df_eeuu)